In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
from technicals.indicators import MACD
from simulation.guru_tester import GuruTester

In [3]:
BUY = 1
SELL = -1
NONE = 0

def apply_signal(row):
    
    if row.direction == BUY and row.mid_l > row.EMA_100:
        return BUY
    if row.direction == SELL and row.mid_h < row.EMA_100:
        return SELL
    return NONE        

In [4]:
pair = "EUR_USD"
df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [5]:
df_an = MACD(df_an)
df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

In [6]:
df_an.head(1)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,MACD,SIGNAL,HIST,EMA_100
0,2016-01-13 03:00:00+00:00,450,1.0829,1.08324,1.08247,1.0825,1.08282,1.08316,1.0824,1.08243,1.08297,1.08331,1.08254,1.08258,-0.001062,-0.000931,-0.000131,1.086813


In [8]:
def apply_cross(row):
    if row.macd_delta > 0 and row.macd_delta_prev < 0:
        return BUY
    if row.macd_delta < 0 and row.macd_delta_prev > 0:
        return SELL
    return NONE

In [9]:
df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
df_an['direction'] = df_an.apply(apply_cross, axis=1)

In [13]:
df_an.direction.value_counts()

direction
 0    34175
 1     1473
-1     1472
Name: count, dtype: int64

In [14]:
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,ask_h,ask_l,ask_c,MACD,SIGNAL,HIST,EMA_100,macd_delta,macd_delta_prev,direction
0,2016-01-13 03:00:00+00:00,450,1.08290,1.08324,1.08247,1.08250,1.08282,1.08316,1.08240,1.08243,...,1.08331,1.08254,1.08258,-0.001062,-0.000931,-0.000131,1.086813,-0.000131,NaN,0
1,2016-01-13 04:00:00+00:00,239,1.08247,1.08314,1.08243,1.08280,1.08239,1.08307,1.08235,1.08272,...,1.08321,1.08251,1.08287,-0.001075,-0.000960,-0.000115,1.086722,-0.000115,-0.000131,0
2,2016-01-13 05:00:00+00:00,518,1.08279,1.08352,1.08272,1.08294,1.08272,1.08344,1.08266,1.08287,...,1.08360,1.08279,1.08301,-0.001062,-0.000981,-0.000082,1.086635,-0.000082,-0.000115,0
3,2016-01-13 06:00:00+00:00,355,1.08290,1.08379,1.08290,1.08332,1.08283,1.08373,1.08282,1.08324,...,1.08385,1.08297,1.08341,-0.001009,-0.000986,-0.000023,1.086560,-0.000023,-0.000082,0
4,2016-01-13 07:00:00+00:00,1647,1.08332,1.08366,1.08134,1.08162,1.08324,1.08358,1.08126,1.08155,...,1.08375,1.08142,1.08169,-0.001092,-0.001007,-0.000085,1.086448,-0.000085,-0.000023,0


In [4]:
def run_pair(pair):
    df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an = RSI(df_an)
    df_an = apply_patterns(df_an)
    df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
            'ENGULFING', 'direction', 'EMA_200', 'RSI_14' ]
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True
    )
    
    gt.run_test()
    return gt.df_results
    

In [5]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p)))

prepare_data...
run_test...
Result: -2065.5
prepare_data...
run_test...
Result: -915.0
prepare_data...
run_test...
Result: -1109.5
prepare_data...
run_test...
Result: -1200.5


In [6]:
for r in res:
    print(r['pair'], r['res'].result.sum())

AUD_NZD -2065.5
EUR_USD -915.0
GBP_JPY -1109.5
USD_CAD -1200.5
